# Labo 6 : désambiguïsation lexicale (Word Sense Disambiguation)
Par Vincent Guidoux et Nathan Gonzalez Montes

## Objectif et informations

Comparer deux méthodes de désambiguïsation du sens des mots en contexte (tâche notée WSD), l’une utilisant l’algorithme de Lesk simplifié, et l’autre utilisant word2vec.

Le labo utilisera une ressource contenant 2'369 occurrences du mot anglais « interest » annotées chacune avec le sens du mot dans le contexte respectif. Chacune des deux méthodes fonctionne selon le même principe général : comparer le contexte de l’occurrence avec les définitions des sens, et choisir la définition la plus proche. L’algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec peut la calculer comme la similarité de deux vecteurs (somme des vecteurs de mots).

Dans une première approche, qui ne nécessite pas de programmation, on vous demande de comparer une occurrence de chaque sens. Dans une seconde approche, qui nécessite un peu de programmation en Python, vous évaluerez les deux systèmes sur l’ensemble des 2'369 occurrences.

## Étapes

### 1. Le fichier de données se trouve à http://www.d.umn.edu/~tpederse/data.html – chercher « interest » vers la fin de la page, et prendre le fichier marqué « local copy ». Quel est le format de ce fichier et comment sont annotés les sens ?

Le fichier est au format texte, son extension est `.acl94`

Exemple d'une ligne du fichier :

`[ longer/JJR maturities/NNS ] are/VBP thought/VBN to/TO indicate/VB [ declining/VBG interest_6/NN rates/NNS ] because/IN [ they/PP ] permit/VBP [ portfolio/NN managers/NNS ] to/TO retain/VB relatively/RB [ higher/JJR rates/NNS ] for/IN [ a/DT longer/JJR period/NN ] ./.`

### 2. Bien lire le fichier README associé. Quelles sont les définitions des six sens de « interest » annotées dans les données ? De quel dictionnaire viennent-elles ? Où se trouve-t-il en ligne ?

1. Readiness to give attention
2. Quality of causing attention to be given to
3. Activity, etc. that one gives attention to
4. Advantage, advancement or favor
5. A share in a company or business
6. Money paid for the use of money

Du dictionnaire LDOCE. Ici: [Longman Dictionary of Contemporary English | LDOCE
](https://www.ldoceonline.com/)


### 3. Consulter WordNet en ligne et identifier les définitions correspondant aux six sens annotés dans les données. En les combinant éventuellement avec les résultats du (2), écrivez une liste de mots « pleins » (i.e. sans stopwords) pour chaque définition.

### Noun

1. A sense of concern with and curiosity about someone or something, **involvement**: *"an interest in music"*

2. The power of attracting or holding one's attention (because it is unusual or exciting etc.), **interestingness**: *"they said nothing of great interest"; "primary colors can add interest to a room"e or something) "an interest in music"*

3. A diversion that occupies one's time and thoughts (usually pleasantly), **pastime, pursuit**: *"sailing is her favorite pastime"; "his main pastime is gambling"; "he counts reading among his interests"; "they criticized the boy for his limited pursuits"*

4. A reason for wanting something done, **sake**: *"for your sake"; "died for the sake of his country"; "in the interest of safety"; "in the common interest"*

5. (law) A right or legal share of something; a financial involvement with something, **stake**: *"they have interests all over the world"; "a stake in the company's future"*

6. A fixed charge for borrowing money; usually a percentage of the amount borrowed: *"how much interest do you pay on your mortgage?"*

(usually plural) a social group whose members control some field of activity and who have common aims, interest group: *"the iron interests stepped up production"*

### Verb

Excite the curiosity of; engage the interest of

**Concern, occupy, worry**, be on the mind of: "I worry about the second Germanic consonant shift"

**Matter to**, be of importance or consequence: "This matters to me!"

1. Readiness, give, attention, sense, concern, curiosity, someone, something, involvement
2. Qualitiy, causing, attraction, be, given, power, attracting, holding, one's, attention, because, unusual, exciting
3. Activity, one, gives, attention, diversion, occupies, one's, time, thoughts, usually, pleasantly, pastime, pursuit

### 4. Considérez la première occurrence de « interest » dans les données, et notez les mots « pleins » qui entourent « interest » (p.ex. toute la phrase). Combien de mots y’a-t-il en commun avec chacune des six définitions ? Quel est donc le sens le plus probable ?

### 5. En réutilisant le modèle de word2vec entraîné sur Google News fourni par Gensim, calculer les similarités (cosinus) entre les mots du contexte de la première occurrence (somme des vecteurs de chaque mot) et chacune des six définitions. Quel est donc le sens le plus probable ?

**Suggestion** : chercher dans la documentation word2vec comment obtenir très facilement la similarité entre deux ensembles de mots.

In [5]:
import numpy
import nltk
import time
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

w2v_vectors = KeyedVectors.load_word2vec_format("word2vec-google-news-300.gz", binary=True)

In [6]:
similarities = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))

print("Pearson correlation coefficient with 2-tailed p-value : ", similarities[0])
print("\n")
print("Spearman rank-order correlation coefficient between the similarities from the dataset and the similarities produced by the model itself, with 2-tailed p-value : ", similarities[1])
print("\n")
print("The ratio of pairs with unknown words : ", similarities[2])
# similarity = w2v_vectors.similarity("present", "past")
# print("Opposite (similarity): {:.3f}".format(similarity))

Pearson correlation coefficient with 2-tailed p-value :  (0.62387734233188386, 1.7963265731961865e-39)


Spearman rank-order correlation coefficient between the similarities from the dataset and the similarities produced by the model itself, with 2-tailed p-value :  SpearmanrResult(correlation=0.65892158880092877, pvalue=2.5346056459149263e-45)


The ratio of pairs with unknown words :  0.0


In [13]:
analogy_scores = w2v_vectors.evaluate_word_analogies(datapath('interest-original.txt'))

print("Le score d'évaluation global sur l'ensemble de l'évaluation : ", analogy_scores[0])
print("\n")
print("Résultats ventilés par section du jeu d’évaluations. Chaque dict contient le nom de la section sous la clé ‘section’, ainsi que des listes de 4 tuples de mots correctement et incorrectement prédits sous les clés ‘correct’ et ‘incorrect’ : ")
print("\n")

for section in analogy_scores[1]:
    print("section \"{section}\" : ".format(section=section["section"]))
    print("number of correct : ", len(section["correct"]))
    print("number of incorrect : ", len(section["incorrect"]))
    print("\n")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Nathan\\Anaconda3\\lib\\site-packages\\gensim\\test\\test_data\\interest-original.txt'

### 6. Approche manuelle : appliquer la procédure précédente à une occurrence de « interest » pour chaque sens possible (six sens, donc six mots). Combien sur les six sont correctement désambiguïsés par l’algorithme de Lesk, et combien par la méthode word2vec ?

### 7. Approche automatique : implémenter un court programme qui applique la méthode décrite ci-dessus à chacune des 2'369 occurrences de « interest » dans le corpus du test.

Il faut donc : extraire les mots voisins de chaque occurrence, compter le nombre de mots en commun avec chaque définition, et choisir le sens qui maximise ce nombre (Lesk) ; également, calculer la similarité word2vec entre le contexte et chaque définition, et choisir le sens qui la maximise.

Quelles sont les proportions respectives de bonnes réponses de Lesk et de word2vec ?
Pouvez-vous les calculer également pour chaque catégorie ?

### 8. Comment se comparent vos scores avec ceux publiés dans l’article de Pedersen (2000) disponible ici : http://www.aclweb.org/anthology/A00-2009 ?

Merci d’envoyer votre notebook Jupyter par email au professeur et à l’assistant, avant le vendredi 17 mai à 23h59.